In [1]:
### Libraries
import pandas as pd
import numpy as np
import altair as alt
import janitor
from datetime import datetime, timedelta
import calendar
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
alt.renderers.enable('default')
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

import altair as alt
#nltk.download()
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
###Importing data

In [3]:
manual_extraction = pd.read_excel('../../../../data/clean_data/manual_extraction_template.xlsx').clean_names()

In [4]:
manual_extraction

,employee_code,employee_name,found,work1_title_,work1_company,work1_time_,work2_title_,work2_company,work2_time_,work3_title_,...,education2_concentration_,education2_country,education3_school_,education3_degree_,education3_concentration_,education3_country,education4_school_,education4_degree_,education4_concentration_,education4_country
0,BDP,"Abdulali, Kaizar",clara,Sales Manager (Samsung Experience Store),Glentel Inc.,Nov 2015 - Present,Sales Manager (Wireless Wave & T-Booth),Glentel Inc.,Aug 2010 - Nov 2015,Sales Associate (Wireless Wave & T-Booth),...,Economic,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F6Q,"Avila-Gil, Mario",clara,Sales Manager,Glentel/ Wireless ETC,NaN,Customer Service Consultant,Cityfone/ Rogers Communications,Nov 2009 - Sep 2011,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MRN,"Amini, Kamyar",clara,Sales Associate/ Educator,THE BRICK,2008-2010,Reception/Cashier,CHAIRMANS MANUFACTURING,2006 - 2008,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MXO,"Awan, Shoaib",clara,Team Leader,TRAVELEX GROUP CANADA,Nov 2014 - Present,Support Trainer,TRAVELEX GROUP CANADA,Jan 2013 - Oct 2014,ATM Coordinator / Operations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N0Y,"Al-Khulaidy, Maha",clara,Sales Associate,Sportcheck FGL Sports,February 2019 - Present,Multi-Channel Sales Associate,Best Buy,Nov 2017 - Jan 2019,Customer Service Consultant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,MUD,"Ryoo, Sungtak",thomas,Technician,uBreakiFix,July 2018 - Present,Production Worker,Toyota,April 2018 - July 2018,Manager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285,L46,"Kapasi, Nikunj",thomas,ECOMMERCE ANALYST,DOT LIGHTING,Jan 2019 - Present,SALES ASSOCIATE,TBOOTH WIRELESS,Feb 2017 - Jan 2019,E-COMMERCE COORDINATOR,...,Science,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,N87,"Mcinnis, Peter",thomas,Order Selector,LOBLAWS,November 2017 - PRESENT,Manager,LINEN CHEST,February 2015 - March 2018,Customer Service Representative,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,ND9,"Gamble, Justin",thomas,Cashier,Save On Foods,March 2017 - Present,Photographer/Illustrator,Gamble Visuals,June 2016 - Present,Sales Associate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
manual_extraction.columns

Index(['employee_code', 'employee_name', 'found', 'work1_title_',
       'work1_company', 'work1_time_', 'work2_title_', 'work2_company',
       'work2_time_', 'work3_title_', 'work3_company', 'work3_time_',
       'work4_title_', 'work4_company', 'work4_time_', 'work5_title_',
       'work5_company', 'work5_time_', 'work6_title_', 'work6_company',
       'work6_time_', 'work7_title_', 'work7_company', 'work7_time_',
       'education1_school_', 'education1_degree_', 'education1_concentration_',
       'education1_country_', 'education2_school_', 'education2_degree_',
       'education2_concentration_', 'education2_country', 'education3_school_',
       'education3_degree_', 'education3_concentration_', 'education3_country',
       'education4_school_', 'education4_degree_', 'education4_concentration_',
       'education4_country'],
      dtype='object')

In [6]:
### Cleaning Academic Background

In [7]:
def preprocess(text): 
    
    text = str(text)
    text = text.lower()
    text = text.strip()
    text = re.sub('\W+',' ', text)
    text = re.sub("[\d-]", '', text)
    
    # Replace a sequence of whitespaces by a single whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove other strange characters
    text = re.sub(r'''[\*\~]+''', "", text)
    
    #hard code changes 
    if text == 'nan': text = "not_specified"
    if text == 'art': text = 'arts'
    if text == 'liberal arts': text = 'arts'
    if text == 'arts marketing': text = 'marketing'
    if text == 'arts psychology': text = 'psychology'
    if text == 'general arts': text = 'arts'
    
    #if they list two take the first degree
    text = re.split(" and ", text)[0]
    
    #replace word 
    text = re.sub("not mention.+", "not_specified", text)
    text = re.sub("advanced ", '', text)
    text = re.sub("degree", '', text)
    text = re.sub("certificate", '', text)
    text = re.sub("administrat", '', text)
    text = re.sub("management", '', text)
    text = re.sub("progress", '', text)
    text = re.sub("mba", 'business', text)
    text = re.sub("bachelor", '', text)
    text = re.sub("commerce", 'business', text)
    text = re.sub("hr", 'human resource', text)
    text = re.sub("mortgage", 'finance', text)
    text = re.sub("securities", 'finance', text)
    text = re.sub("radio", 'audio', text)
    text = re.sub("tv ", 'television', text)
    text = re.sub('television', 'film', text)
    text = re.sub('health care', 'healthcare', text)
    text = re.sub('hospital ', 'healthcare', text)
    text = re.sub('computing', 'computer', text)
    text = re.sub('techonology', 'technology', text)
    text = re.sub('socialogy', 'sociology', text)
    text = re.sub('child', 'education', text)
    text = re.sub('office', 'general', text)
    text = re.sub('general arts', 'arts', text)
    text = re.sub('public', 'general', text)
    text = re.sub('o s s d', 'general', text)
    text = re.sub('informatics security', 'computer', text) 
    text = re.sub("bank", 'business', text) 
    
    #group degree based off of key words
    if re.search("law", text): text = "law"
    if re.search("dental", text): text = "dental"
    if re.search("financ", text): text = "finance"
    if re.search("account", text): text = "accounting"
    if re.search("human resource", text): text = "human resource"
    if re.search("marketing", text): text = "marketing"
    if re.search("business", text): text = "business"
    if re.search("audio", text): text = "audio technician"
    if re.search("film", text): text = "film production"
    if re.search('healthcare', text): text = "healthcare"
    if re.search('counsel', text): text = "healthcare"
    if re.search('engineering', text): text = "engineering"
    if re.search('communication', text): text = "communication"
    if re.search('education', text): text = 'education'
    if re.search('kinesiology', text): text = 'kinesiology'
    if re.search('computer', text): text = 'computer systems'
    if re.search('software', text): text = 'computer systems'
    if re.search('police', text): text = 'law'
    if re.search('science', text): text = 'science'
    if re.search('english', text): text = 'english'
    if re.search('paralegal', text): text = 'law'
    if re.search('social service', text): text = 'arts'
    if re.search('esthetician', text): text = 'health'
    if re.search('fitness', text): text = 'health'
    if re.search('carpent', text): text = 'blue collar'
    if re.search('plumb', text): text = 'blue collar'
    if re.search('electric', text): text = 'blue collar'
    if re.search('crane', text): text = 'blue collar'
    if re.search('media', text): text = 'interactive arts and technology'
    if re.search('technology', text): text = 'interactive arts and technology'
    if re.search('game', text): text = 'interactive arts and technology'
    if re.search('entertainment', text): text = 'interactive arts and technology'
    if re.search('information', text): text = 'interactive arts and technology'
    if re.search('digital', text): text = 'interactive arts and technology'
    if re.search('medica', text): text = "healthcare"
    if re.search('nurs', text): text = 'healthcare'
    if re.search('pharma', text): text = 'healthcare'
    if re.search('linguist', text): text = 'english'
    if re.search('translation', text): text = 'arts'
    if re.search('general', text): text = 'general'
    if re.search('photo', text): text = 'interactive arts and technology'
    if re.search('screen', text): text = 'interactive arts and technology'
    if re.search('profession', text): text = 'arts'
    if re.search('liberal', text): text = 'arts'
    if re.search('graphic', text): text = 'interactive arts and technology'
    
    
    if text == 'not mention': text = "not_specified"
    
    
    #text = re.sub("arts ", '', text)
    
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text

In [8]:
#could have made this more efficent
def concentration_labeling(con):
    con = str(con)
    if any(x in con for x in ['business']):
        label = "business"
    elif any(x in con for x in ['science']):
        label = "science"
    elif con == 'arts':
        label = "arts"
    elif any(x in con for x in ['dental']):
        label = "dental"
    elif con == 'interactive arts and technology': 
        label = 'interactive arts and technology'
    elif any(x in con for x in ['computer systems']):
        label = 'computer systems'
    elif any(x in con for x in ['hospitality']):
        label = 'hospitality'
    elif any(x in con for x in ['sociology']):
        label = 'sociology'
    elif any(x in con for x in ['finance']):
        label = 'finance'
    elif any(x in con for x in ['marketing']):
        label = 'marketing'
    elif any(x in con for x in ['law']):
        label = 'law'
    elif any(x in con for x in ['engineering']):
        label = 'engineering'
    elif any(x in con for x in ['psychology']):
        label = 'psychology'
    elif any(x in con for x in ['general']):
        label = 'general'
    elif any(x in con for x in ['healthcare']):
        label = 'healthcare'
    elif any(x in con for x in ['human resource']):
        label = 'human resource'
    elif any(x in con for x in ['physics']):
        label = 'physics'
    elif any(x in con for x in ['accounting']):
        label = 'accounting'
    elif any(x in con for x in ['kinesiology']):
        label = 'kinesiology'
    elif any(x in con for x in ['audio technician']):
        label = 'audio technician'
    elif any(x in con for x in ['education']):
        label = 'education'
    elif any(x in con for x in ['blue collar']):
        label = 'blue collar'
    elif any(x in con for x in ['communication']):
        label = 'communication'
    elif any(x in con for x in ['english']):
        label = 'english'
    elif any(x in con for x in ['criminology']):
        label = 'criminology'
    elif any(x in con for x in ['statistics']):
        label = 'statistics'
    elif any(x in con for x in ['economics']):
        label = 'economics'
    elif any(x in con for x in ['not_specified']):
        label = 'not_specified'
    else:
        label = "other" 
    return label

In [9]:
def background_label_generator(txt):
    txt = preprocess(txt)
    txt = concentration_labeling(txt)
    return txt

In [10]:
manual_extraction['education1_clean_background'] = manual_extraction.education1_concentration_.apply(lambda x: background_label_generator(x))
manual_extraction['education2_clean_background'] = manual_extraction.education2_concentration_.apply(lambda x: background_label_generator(x))
manual_extraction['education3_clean_background'] = manual_extraction.education3_concentration_.apply(lambda x: background_label_generator(x))
manual_extraction['education4_clean_background'] = manual_extraction.education4_concentration_.apply(lambda x: background_label_generator(x))

In [11]:
manual_extraction.rename(columns={'education2_country':'education2_country_','education3_country':'education3_country_','education4_country':'education4_country_'},inplace=True)

In [12]:
cols = list(['a','b'])

In [13]:
cols

['a', 'b']

In [14]:
for i in cols:
    print(i)

a
b


In [15]:
for n in range(7):
    print(n+1)

1
2
3
4
5
6
7


In [16]:
def create_dict_edu(df_temp):
    for n in range(7):
        for t in list(['title','company','time']):
            col_name = "work"+str(n)+"_"+t
            print(col_name)
            print(manual_extraction[col_name])

In [17]:
manual_extraction.columns

Index(['employee_code', 'employee_name', 'found', 'work1_title_',
       'work1_company', 'work1_time_', 'work2_title_', 'work2_company',
       'work2_time_', 'work3_title_', 'work3_company', 'work3_time_',
       'work4_title_', 'work4_company', 'work4_time_', 'work5_title_',
       'work5_company', 'work5_time_', 'work6_title_', 'work6_company',
       'work6_time_', 'work7_title_', 'work7_company', 'work7_time_',
       'education1_school_', 'education1_degree_', 'education1_concentration_',
       'education1_country_', 'education2_school_', 'education2_degree_',
       'education2_concentration_', 'education2_country_',
       'education3_school_', 'education3_degree_', 'education3_concentration_',
       'education3_country_', 'education4_school_', 'education4_degree_',
       'education4_concentration_', 'education4_country_',
       'education1_clean_background', 'education2_clean_background',
       'education3_clean_background', 'education4_clean_background'],
      dtype='o

In [18]:
#myDict_out = dict()
list_school = list()
list_degree = list()
list_concentration = list()
list_country = list()
for n in range(4):
    #myDict_in = dict()
    for t in list(['school_','degree_','concentration_','country_']):
        col_name = "education"+str(n+1)+"_"+t
        current_col = manual_extraction[col_name]
        for i in current_col:
            if len(str(i))<2 or str(i)=="nan" or "mention" in str(i):
                i = "unknown"
            if t == 'school_':
                list_school.append(i)
            elif t == 'degree_':
                list_degree.append(i)
            elif t == 'concentration_':
                list_concentration.append(i)
            elif t == "country_":
                list_country.append(i)

In [19]:
## Should we tokenize or use raw inputs?

In [20]:
def clean_token(doc):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation+"·"+"−"+"§"+"•"+"–"+"––"+"✓")
    lemma = WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #doc_clean = normalized.split() 
    return normalized    

In [21]:
## Degrees

In [22]:
degree_df = pd.DataFrame({"degrees":list_degree})

In [23]:
degree_df['clean_degree'] = degree_df.degrees.apply(lambda x: clean_token(x))

In [24]:
degree_df.sample(n=15)

,degrees,clean_degree
555,Diploma,diploma
657,High School,high school
229,Bachelor - Sep 2016-Current,bachelor sep 2016current
908,unknown,unknown
1014,unknown,unknown
90,Diploma,diploma
986,unknown,unknown
126,unknown,unknown
70,Diploma,diploma
933,unknown,unknown


In [25]:
def degree_labeling(deg):
    code = 2
    label = "other"
    if any(x in deg for x in ['phd']):
        code = 8
        label = "phd"
    elif any(x in deg for x in ['master','msc']):
        code = 7
        label = "master"
    elif any(x in deg for x in ['postgrad']):
        code = 6
        label = "postgrad"
    elif any(x in deg for x in ['bac','bsc']):
        code = 5
        label = "bachelor"
    elif any(x in deg for x in ['high','second','ba','grade','school']):
        code = 3
        label = "highschool"
    elif any(x in deg for x in ['prof','certif','techn','continu','advance','college','program','diploma','graduate']):
        code = 4
        label = "certificate"
    elif "unknown" in deg:
        code = 1
        label = "not_specified"
    return [code,label]

In [26]:
def clean_token(str_value,tpe='t'):
    if len(str(str_value))<2 or str(str_value)=="nan" or ("mention" in str(str_value)):
        str_value = "unknown"
    # "Other" is not considered a stopword, because it was generating labelling issues
    stop_list = stopwords.words('english')
    stop_list.remove('other')
    stop = set(stop_list)
    exclude = set(string.punctuation+"·"+"−"+"§"+"•"+"–"+"––"+"✓")
    lemma = WordNetLemmatizer()
    stop_free = " ".join([i for i in str_value.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized 

In [27]:
def edu_dict(manual_df):  
    myDict = dict()
    for n in range(4):
        for t in list(['school_','degree_','clean_background','country_']):
            col_name = "education"+str(n+1)+"_"+t
            i = manual_df[col_name]
            clean_i = clean_token(i)
            #clean_i = str(i)
            myDict.setdefault(t, []).append(clean_i)
    ####Labelling Degrees
    myDict["Degree_Code"] = list(map(lambda x: degree_labeling(x)[0], myDict['degree_']))
    myDict["Degree_Label"] = list(map(lambda x: degree_labeling(x)[1], myDict['degree_']))
    pos_highest_degree = myDict['Degree_Code'].index(max(myDict['Degree_Code']))
    myDict["highest_degree"] = myDict['Degree_Label'][pos_highest_degree]
    myDict["background_highest_degree"] = myDict['clean_background'][pos_highest_degree]
    myDict["country_highest_degree"] = myDict['country_'][pos_highest_degree]    
    return myDict

In [28]:
manual_extraction['highest_degree'] = manual_extraction.apply(lambda x: edu_dict(x)['highest_degree'],axis=1)
manual_extraction['background_highest_degree'] = manual_extraction.apply(lambda x: edu_dict(x)['background_highest_degree'],axis=1)
manual_extraction['country_highest_degree'] = manual_extraction.apply(lambda x: edu_dict(x)['country_highest_degree'],axis=1)

In [29]:
manual_extraction[['highest_degree','background_highest_degree','country_highest_degree']]

,highest_degree,background_highest_degree,country_highest_degree
0,bachelor,business,canada
1,highschool,notspecified,canada
2,bachelor,business,canada
3,highschool,notspecified,unknown
4,bachelor,science,canada
...,...,...,...
284,certificate,law,canada
285,bachelor,science,india
286,not_specified,notspecified,unknown
287,not_specified,notspecified,unknown


In [30]:
train_dataset = pd.read_csv('C:/Users/NIW/Glentel Inc/HR Analytics - Capstone Data/ubc_mds_team_share/02_eda_data/output/train_dataset.csv',index_col = 0)

In [31]:
train_dataset[['employee_code','hp_class']]

,employee_code,hp_class
2825,MZE,0.0
2639,MRU,0.0
1361,LAP,0.0
2799,MYD,1.0
668,JJB,0.0
...,...,...
2659,MTI,0.0
3122,N8M,0.0
2839,MZX,0.0
2994,N4J,0.0


In [32]:
var_distr = pd.merge(manual_extraction,train_dataset[['employee_code','hp_class']],on='employee_code',how='inner')

In [33]:
var_distr.groupby('highest_degree').agg({'employee_code':"count",'hp_class':'mean'}).sort_values(by="employee_code",ascending = False)

,employee_code,hp_class
highest_degree,,
bachelor,101,0.306931
certificate,60,0.200000
highschool,59,0.118644
not_specified,32,0.312500
master,26,0.269231
other,8,0.375000
postgrad,2,0.000000
phd,1,0.000000


In [34]:
var_distr.groupby('background_highest_degree').agg({'employee_code':"count",'hp_class':'mean'}).sort_values(by="employee_code",ascending = False)

,employee_code,hp_class
background_highest_degree,,
notspecified,61,0.229508
business,51,0.313725
general,32,0.156250
computer system,16,0.250000
law,12,0.083333
engineering,11,0.181818
other,10,0.100000
art,10,0.200000
finance,9,0.555556


In [35]:
var_distr.groupby('country_highest_degree').agg({'employee_code':"count",'hp_class':'mean'}).sort_values(by="employee_code",ascending = False)

,employee_code,hp_class
country_highest_degree,,
canada,178,0.207865
unknown,56,0.250000
india,28,0.321429
pakistan,5,0.200000
usa,3,0.000000
england,3,1.000000
indore,2,1.000000
philippine,2,0.000000
uk,2,0.500000


In [36]:
var_distr.groupby(['highest_degree','country_highest_degree']).agg({'employee_code':"count",'hp_class':'mean'})

employee_code  hp_class
highest_degree country_highest_degree                         
bachelor       brazil                              1  0.000000
               canada                             63  0.253968
               china                               1  1.000000
               england                             2  1.000000
               india                              19  0.368421
               philippine                          2  0.000000
               punjab                              1  0.000000
               syria                               1  0.000000
               taiwan                              1  0.000000
               turkey                              1  0.000000
               u                                   1  0.000000
               uk                                  1  1.000000
               unknown                             6  0.500000
               vietnam                             1  1.000000
certificate    canada                             55  0.181818
               india                               1  1.000000
               unknown                             3  0.333333
               usa                                 1  0.000000
highschool     canada                             38  0.157895
               india                               1  0.000000
               unknown                            20  0.050000
master         canada                              7  0.142857
               dubai                               1  0.000000
               england                             1  1.000000
               india                               7  0.142857
               indore                              2  1.000000
               pakistan                            5  0.200000
               uk                                  1  0.000000
               united kingdom                      1  1.000000
               usa                                 1  0.000000
not_specified  canada                              5  0.200000
               unknown                            27  0.333333
other          canada                              7  0.428571
               usa                                 1  0.000000
phd            canada                              1  0.000000
postgrad       canada                              2  0.000000

In [37]:
manual_extraction[['employee_code','highest_degree','background_highest_degree','country_highest_degree']].to_csv('../../../../data/clean_data/features_01_training_df.csv')

## Separating Dates (Discarded)

In [38]:
def has_sep(time_string):
    time_string = str(time_string)
    if any(x in time_string for x in ['-',' to','–','—','till']):
        return 1
    else:
        return 0

In [39]:
def year_pattern_finder(time_string):
    #iter = re.finditer(r"[\d]{4}", time_string)
    #indices = [m.start(0) for m in iter]
    #labels =
    time_string = str(time_string)
    #print(time_string)
    out = re.findall(r"[a-z]+[\s][\d]{4}",time_string)
    return out

In [40]:
sample_text = "nov 2014-aug 2016"

In [41]:
iter = re.finditer(r"[\d]{4}", sample_text)

In [42]:
for i in iter:
    print(i.start(0))

4
13


In [43]:
def has_sep(time_string):
    time_string = str(time_string)
    if any(x in time_string for x in ['-',' to','–','—','till']):
        return 1
    else:
        return 0

In [44]:
def date_structuring(time_string):
    myDict = dict() 
    time_string = str(time_string)
    time_string = time_string.lower()
    if has_sep(time_string) == 1:
        split_string = re.split('-| to|–|—|till',time_string)
        myDict.setdefault('start', []).append(split_string[0])
        myDict.setdefault('end', []).append(split_string[1])
    else:
        myDict.setdefault('start', []).append(time_string)
        myDict.setdefault('end', []).append(time_string)
    return myDict

In [45]:
list_date = list()
for t in list(['start','end']):
    current_col = time_df[t]
    for i in current_col:
        i = str(i)
        if len(str(i))<2 or str(i)=="nan" or "mention" in str(i) or "year" in i:
            i = "unknown"
        list_date.append(i)

NameError: name 'time_df' is not defined

In [ ]:
single_date_df = pd.DataFrame({"list_date":list_date})
single_date_df.sample(10)

# Helping with Industry Categorization

In [ ]:
### Industry sector

In [ ]:
import textdistance

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
import textdistance

## Method 2

In [643]:
# Company data
resume_info = pd.read_excel('../../../../data/clean_data/manual_extraction_template.xlsx')
# List of food_service_retailers
foodservice_df = pd.read_csv('../../../../data/clean_data/food_retail_list.csv')
# list of retailers
retailers_df = pd.read_csv('../../../../data/clean_data/retail_list.csv')

In [62]:
additional_list_1 = ['a & w', 'a&w', 'chuck e. cheese', 'wendy', "wendy's", 'touchdown willies', 
                   "touchdown willy's tap & grill", 'tony roma’s fallsview', 'tony roma’s', "denny's", 
                   'denneys', 'hard rock cafe', 'hard rock café', 'pickle barrel', 'hummingbird pub', 
                   'starbucks', 'the london club', 'the nutty chocolatier', 'wild wings', 'forrat’s lounge', 
                   'costa blanca', 'harveys & swiss chalet', "tucker's market place", 'bier markt', 'blco foods', 
                   'hard rock cafe']

In [64]:
foodservice_df.head(5)

,restaurant_retailers
0,241 Pizza
1,A&W
2,A&W Canada
3,Arby's
4,Baker's Dozen Donuts


## Helper functions

In [671]:
def clean_txt_f(txt):
    txt = str(txt)
    punct_list = list(string.punctuation+"·"+"−"+"§"+"•"+"–"+"––"+"✓")
    punct_list.remove("&")
    exclude = set(punct_list)
    clean_txt = "".join([i for i in txt.lower() if i not in exclude])
    clean_txt = re.sub(r'\s+', ' ', clean_txt)
    return clean_txt

In [66]:
list_1 = list()
for i in foodservice_df['restaurant_retailers']:
    list_1.append(clean_txt_f(i))
##Removing duplicates
list_1 = list(dict.fromkeys(list_1))

In [67]:
list_2 = list()
for i in additional_list_1:
    list_2.append(clean_txt_f(i))
list_2 = list(dict.fromkeys(additional_list_1))

In [68]:
key_list = ["restaurant","pizza", "steak", "burger", "grill", "dry bar", "coffee", "donald", "taco", 
                            "catering", "burrito", "snack", "cafe", "pretzels", "mexican", "sweet", "coffee shop",
                            "cuisine", "cafeteria", "chocolat", "chicken"]

In [678]:
def lookup_similar(txt,list_search):
    similarity_scores = list(map(lambda x: textdistance.levenshtein.normalized_similarity(txt,x), list_search))
    pos_max_score = similarity_scores.index(max(similarity_scores))
    max_score = similarity_scores[pos_max_score]
    matched_company = list_search[pos_max_score]
    results = list([max_score,matched_company,pos_max_score])
    return results

In [70]:
def food_label(company_name,clean_comparison_list,clean_additional_list,keyword_list,score_threshold=0.80):
    company_name = clean_txt_f(str(company_name))
    myDict = dict()
    search_1 = lookup_similar(company_name,clean_comparison_list)
    if search_1[0]>score_threshold:
        myDict["food_label"] = 1
        myDict["method"] = 'comparison_list'
        myDict["company_matched"] = search_1[1]
        myDict["similarity_score"] = search_1[0]
        return myDict
    search_2 = lookup_similar(company_name,clean_additional_list)
    if search_2[0]>score_threshold:
        myDict["food_label"] = 1
        myDict["method"] = 'additional_list'
        myDict["company_matched"] = search_2[1]
        myDict["similarity_score"] = search_2[0]
        return myDict
    if any(x in company_name for x in keyword_list):
        myDict["food_label"] = 1
        myDict["method"] = 'keyword_list'
        myDict["company_matched"] = "keyword_na"
        myDict["similarity_score"] = 1
        return myDict
    else:
        myDict["food_label"] = 0
        myDict["method"] = 'not_matched'
        myDict["company_matched"] = "not_matched"
        myDict["similarity_score"] = 0
        return myDict

In [71]:
sample = resume_info[['employee_code','work1_company']]
sample

,employee_code,work1_company
0,BDP,Glentel Inc.
1,F6Q,Glentel/ Wireless ETC
2,MRN,THE BRICK
3,MXO,TRAVELEX GROUP CANADA
4,N0Y,Sportcheck FGL Sports
...,...,...
284,MUD,uBreakiFix
285,L46,DOT LIGHTING
286,N87,LOBLAWS
287,ND9,Save On Foods


In [72]:
sample['food_dict'] = sample.work1_company.apply(lambda x: food_label(x,list_1,list_2,key_list))

C:\Users\NIW\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [73]:
sample

,employee_code,work1_company,food_dict
0,BDP,Glentel Inc.,"{'food_label': 0, 'method': 'not_matched', 'co..."
1,F6Q,Glentel/ Wireless ETC,"{'food_label': 0, 'method': 'not_matched', 'co..."
2,MRN,THE BRICK,"{'food_label': 0, 'method': 'not_matched', 'co..."
3,MXO,TRAVELEX GROUP CANADA,"{'food_label': 0, 'method': 'not_matched', 'co..."
4,N0Y,Sportcheck FGL Sports,"{'food_label': 0, 'method': 'not_matched', 'co..."
...,...,...,...
284,MUD,uBreakiFix,"{'food_label': 0, 'method': 'not_matched', 'co..."
285,L46,DOT LIGHTING,"{'food_label': 0, 'method': 'not_matched', 'co..."
286,N87,LOBLAWS,"{'food_label': 0, 'method': 'not_matched', 'co..."
287,ND9,Save On Foods,"{'food_label': 0, 'method': 'not_matched', 'co..."


In [74]:
sample.food_dict.iloc[0]

{'food_label': 0,
 'method': 'not_matched',
 'company_matched': 'not_matched',
 'similarity_score': 0}

In [75]:
sample['food_flag'] = sample.food_dict.apply(lambda x: x['food_label'])
sample['method'] = sample.food_dict.apply(lambda x: x['method'])
sample['company_matched'] = sample.food_dict.apply(lambda x: x['company_matched'])
sample['similarity_score'] = sample.food_dict.apply(lambda x: x['similarity_score'])

C:\Users\NIW\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\NIW\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\NIW\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [76]:
sample

,employee_code,work1_company,food_dict,food_flag,method,company_matched,similarity_score
0,BDP,Glentel Inc.,"{'food_label': 0, 'method': 'not_matched', 'co...",0,not_matched,not_matched,0.0
1,F6Q,Glentel/ Wireless ETC,"{'food_label': 0, 'method': 'not_matched', 'co...",0,not_matched,not_matched,0.0
2,MRN,THE BRICK,"{'food_label': 0, 'method': 'not_matched', 'co...",0,not_matched,not_matched,0.0
3,MXO,TRAVELEX GROUP CANADA,"{'food_label': 0, 'method': 'not_matched', 'co...",0,not_matched,not_matched,0.0
4,N0Y,Sportcheck FGL Sports,"{'food_label': 0, 'method': 'not_matched', 'co...",0,not_matched,not_matched,0.0
...,...,...,...,...,...,...,...
284,MUD,uBreakiFix,"{'food_label': 0, 'method': 'not_matched', 'co...",0,not_matched,not_matched,0.0
285,L46,DOT LIGHTING,"{'food_label': 0, 'method': 'not_matched', 'co...",0,not_matched,not_matched,0.0
286,N87,LOBLAWS,"{'food_label': 0, 'method': 'not_matched', 'co...",0,not_matched,not_matched,0.0
287,ND9,Save On Foods,"{'food_label': 0, 'method': 'not_matched', 'co...",0,not_matched,not_matched,0.0


In [77]:
sample[sample['food_flag']==1]

,employee_code,work1_company,food_dict,food_flag,method,company_matched,similarity_score
15,LWG,McDonalds,"{'food_label': 1, 'method': 'comparison_list',...",1,comparison_list,mcdonalds,0.900000
26,N8M,PIZZA PIZZA Ltd.,"{'food_label': 1, 'method': 'keyword_list', 'c...",1,keyword_list,keyword_na,1.000000
67,N76,McDonalds,"{'food_label': 1, 'method': 'comparison_list',...",1,comparison_list,mcdonalds,0.900000
83,N6K,Pacific Coast Catering,"{'food_label': 1, 'method': 'keyword_list', 'c...",1,keyword_list,keyword_na,1.000000
88,MOI,Tim Hortons,"{'food_label': 1, 'method': 'comparison_list',...",1,comparison_list,tim hortons,0.916667
108,N0R,Mr. Pretzels,"{'food_label': 1, 'method': 'keyword_list', 'c...",1,keyword_list,keyword_na,1.000000
149,N10,Kami pizza,"{'food_label': 1, 'method': 'keyword_list', 'c...",1,keyword_list,keyword_na,1.000000
150,N5S,Johnny Rocco’s Restaurant,"{'food_label': 1, 'method': 'keyword_list', 'c...",1,keyword_list,keyword_na,1.000000
155,N3V,The Keg,"{'food_label': 1, 'method': 'comparison_list',...",1,comparison_list,the keg,1.000000
163,N65,Heather Brazeau's Catering Service,"{'food_label': 1, 'method': 'keyword_list', 'c...",1,keyword_list,keyword_na,1.000000


## Method 3

In [645]:
master_list = pd.read_csv('../../../../data/clean_data/master_list_sectors1.csv')
master_list.groupby(["Sector"]).agg({'Store':"count"}).sort_values(by="Store",ascending = False).head(10)

,Store
Sector,
Apparel & Clothing,89
Food retailers,41
Software,39
Broadcasting & entertainment,31
Sport & Enterteinment,29
Food Service,27
Telecomunications,23
Mining,22
Convenience,21


In [657]:
## What are all the companies names in the manual extraction?

In [659]:
manual_extraction.columns

Index(['employee_code', 'employee_name', 'found', 'work1_title_',
       'work1_company', 'work1_time_', 'work2_title_', 'work2_company',
       'work2_time_', 'work3_title_', 'work3_company', 'work3_time_',
       'work4_title_', 'work4_company', 'work4_time_', 'work5_title_',
       'work5_company', 'work5_time_', 'work6_title_', 'work6_company',
       'work6_time_', 'work7_title_', 'work7_company', 'work7_time_',
       'education1_school_', 'education1_degree_', 'education1_concentration_',
       'education1_country_', 'education2_school_', 'education2_degree_',
       'education2_concentration_', 'education2_country_',
       'education3_school_', 'education3_degree_', 'education3_concentration_',
       'education3_country_', 'education4_school_', 'education4_degree_',
       'education4_concentration_', 'education4_country_',
       'education1_clean_background', 'education2_clean_background',
       'education3_clean_background', 'education4_clean_background'],
      dtype='o

In [684]:
#myDict_out = dict()
list_company = list()
for n in range(7):
    #myDict_in = dict()
    for t in list(['company']):
        col_name = "work"+str(n+1)+"_"+t
        current_col = manual_extraction[col_name]
        for i in current_col:
            if len(str(i))<2 or str(i)=="nan":
                continue
                i = "unknown"
            if t == 'company':
                list_company.append(i)
            #elif t == 'degree_':
            #    list_degree.append(i)
            #elif t == 'concentration_':
            #    list_concentration.append(i)
            #elif t == "country_":
            #    list_country.append(i)

In [686]:
len(list_company)

1079

In [668]:
master_list.head(5)

,Store,Sector
0,A Buck or Two,Convenience
1,Abercrombie and Fitch,Apparel & Clothing
2,American Eagle Outfitters,Apparel & Clothing
3,Anthropologie,Apparel & Clothing
4,Apple Store,Consumer electronics


In [674]:
master_list['clean_store_name'] = master_list.Store.apply(lambda x: clean_txt_f(x))
master_list.head(5)

,Store,Sector,clean_store_name
0,A Buck or Two,Convenience,a buck or two
1,Abercrombie and Fitch,Apparel & Clothing,abercrombie and fitch
2,American Eagle Outfitters,Apparel & Clothing,american eagle outfitters
3,Anthropologie,Apparel & Clothing,anthropologie
4,Apple Store,Consumer electronics,apple store


In [677]:
store_list = master_list['clean_store_name']
store_sector = master_list['Sector']

In [693]:
def company_industry_label_generator(comp_name,store_list,store_sector,score_threshold = 0.80):
    company_name = clean_txt_f(str(comp_name))
    myDict = dict()
    search_1 = lookup_similar(company_name,store_list)
    if search_1[0]>score_threshold:
        myDict["industry"] = store_sector[search_1[2]]
        myDict["method"] = 'master_list'
        myDict["company_matched"] = search_1[1]
        myDict["similarity_score"] = search_1[0]
        return myDict
    else:
        myDict["industry"] = "unknown"
        myDict["method"] = 'not_matched'
        myDict["company_matched"] = "not_matched"
        myDict["similarity_score"] = 0
        return myDict

In [694]:
temp_results_eval = pd.DataFrame({"Company_Name":list_company})

In [695]:
temp_results_eval['Industy_Label'] = temp_results_eval.Company_Name.apply(lambda x:(company_industry_label_generator(x,store_list,store_sector)))

In [698]:
temp_results_eval.Industy_Label.iloc[0]

{'industry': 'unknown',
 'method': 'not_matched',
 'company_matched': 'not_matched',
 'similarity_score': 0}

In [699]:
temp_results_eval['label'] = temp_results_eval.Industy_Label.apply(lambda x: x['industry'])
temp_results_eval['company_match'] = temp_results_eval.Industy_Label.apply(lambda x: x['company_matched'])

In [706]:
temp_results_eval

,Company_Name,Industy_Label,label,company_match
0,Glentel Inc.,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
1,Glentel/ Wireless ETC,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
2,THE BRICK,"{'industry': 'Specialty retailers', 'method': ...",Specialty retailers,the brick
3,TRAVELEX GROUP CANADA,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
4,Sportcheck FGL Sports,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
...,...,...,...,...
1074,"Indian Health Consultant, Delhi, India","{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
1075,"Lana Tour Ltd., Hanoi, Vietnam","{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
1076,"TNS India Pvt. Ltd., Hyderabad, India","{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
1077,Hard Rock Cafe,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched


In [715]:
results =  temp_results_eval.groupby(['label']).agg({'Company_Name':'count'}).sort_values(by='Company_Name',ascending = False)
results['perc'] = results.apply(lambda x:100 * x / float(x.sum()))
results

,Company_Name,perc
label,,
unknown,928,86.005561
Consumer electronics,44,4.077850
Department Store,21,1.946247
Apparel & Clothing,20,1.853568
Telecomunications,20,1.853568
Convenience,8,0.741427
Food retailers,7,0.648749
Food Service,6,0.556070
Specialty retailers,6,0.556070


In [726]:
temp_results_eval[temp_results_eval['label']=='unknown'].sample(n=10)

,Company_Name,Industy_Label,label,company_match
458,Dogree Fashions Inc,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
608,FRESHLY SQUEEZED,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
843,Home Depot Canada,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
926,WELLS FARGO,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
745,Bench,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
415,Silver city Theater,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
609,"Petals For You, Panchkula India","{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
524,Big Eagle Services,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
24,Laleeqe Ltd,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched
355,Jersey Mike’s Subs,"{'industry': 'unknown', 'method': 'not_matched...",unknown,not_matched


# More feature extraction

In [46]:
cleaned_english_resume = pd.read_csv('../../../../data/clean_data/05182020_cleaned_english_resumes_V1.0.csv',index_col = 0)
training_df =  pd.read_csv('../../../../data/V2 data 5.1.2020/train_dataset.csv',index_col = 0)
cleaned_english_resume.columns

Index(['employee_name', 'employee_code', 'store', 'raw_resume', 'resume_text',
       'resume_bline', 'language', 'file_type', 'clean_text'],
      dtype='object')

In [47]:
data = pd.merge(cleaned_english_resume[['employee_code','raw_resume','resume_text','resume_bline','clean_text']],training_df[['employee_code','hp_class']],on="employee_code",how = "inner")

In [48]:
data

,employee_code,raw_resume,resume_text,resume_bline,clean_text,hp_class
0,N3O,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Skills Experience MELIKA ABDI Carmelo Avenue ...,"['', '', '', '', '', '', '', '', '', '', '', '...",Skills Experience Carmelo Avenue Ph ability wo...,0.0
1,BDP,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,KAIZAR ABDULALI kaizar83@gmail.com | 780 990-...,"['', '', '', '', '', '', '', '', '', '', '', '...",thank consideration role Project Manager Busin...,0.0
2,MZV,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Michael Abrams 5214 Gerrish Street, Apt 221 C...","['', '', '', '', '', '', '', '', '', '', '', '...",Gerrish Apt entrepreneur new customer acquisit...,0.0
3,N62,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"WALI ABRO 4 Montpelier Street, Brampton, ON |...","['', '', '', '', '', '', '', '', '', '', '', '...",Montpelier OVERVIEW Languages Fluent English S...,0.0
4,N7R,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Communicating Your Value Proposition: Nabil A...,"['', '', '', '', '', '', '', '', '', '', '', '...",communicate value proposition Beryl Private ob...,1.0
...,...,...,...,...,...,...
283,NCD,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Mikaela Yohannes ০Phone: 226-926-3607 ০ E-Mai...,"['', '', '', '', '', '', '', '', '', '', '', '...",email skill Strong write verbal interpersonal ...,0.0
284,MQF,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Weiwei Zhu Home Address: 160 chapel street Ot...,"['', '', '', '', '', '', '', '', '', '', '', '...",Weiwei chapel street AIM outgoing engage perso...,1.0
285,N7T,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Resume Contact www.linkedin.com/in/jonaid- ni...,"['', '', '', '', '', '', '', '', '', '', '', '...",Resume Contact LinkedIn Skills Customer Servic...,1.0
286,KRG,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Resume Contact www.linkedin.com/in/sandeep- k...,"['', '', '', '', '', '', '', '', '', '', '', '...",Resume Contact LinkedIn Skills sale Management...,1.0


In [49]:
### identifying languages

In [615]:
def identify_languages(resume_text):
    myDict = dict()
    spoken_lang = list()
    lang_list = ['spanish','french','german','punjabi','hindi','urdu','arabic','mandarin','dari','japanese','filipino','tamil','cantonese','russian']
    #'tagalog' --removed for possible double counting with filipino
    #lang_list = ['fluent','lang']
    txt = str(resume_text)
    txt = resume_text.lower()
    spoken_lang.append('english')
    for x in lang_list:
        if x in txt:
            spoken_lang.append(x)
        #Creating languages flags
        new_col_name = x +"_flag"
        
    #removing duplicate languages (mentioned more than once in resume)
    spoken_lang = list(dict.fromkeys(spoken_lang))
    no_languages_spoken = len(spoken_lang)
    if no_languages_spoken > 2:
        
        trilingual_flag = 1
    else:
        trilingual_flag = 0
    myDict['spoken_languages'] = spoken_lang
    myDict['no_lang'] = no_languages_spoken
    myDict['trilingual_flag'] = trilingual_flag  
    return myDict

### Additional suggested Glentel Features

In [617]:
def goal_record(txt):
    txt = str(txt).lower()
    if any(x in txt for x in ['target','achiev','award']):
        flag = 1
    else:
        flag = 0
    return flag

In [618]:
def sales_customer_base_exp(txt):
    txt = str(txt).lower()
    if any(x in txt for x in ['sales']) and any(x in txt for x in ['customer']):
        flag = 1
    else:
        flag = 0
    return flag

In [619]:
def volunteer_exp(txt):
    txt = str(txt).lower()
    if any(x in txt for x in ['volun']):
        flag = 1
    else:
        flag = 0
    return flag

In [620]:
def problem_solver(txt):
    txt = str(txt).lower()
    if any(x in txt for x in ['solution','answers','analysis','decision']):
        flag = 1
    else:
        flag = 0
    return flag

In [621]:
def sports_mention(txt):
    txt = str(txt).lower()
    if re.search(r"\bsport", txt):
        flag = 1
    else:
        flag = 0
    return flag

In [622]:
def communication_skills(txt):
    txt = str(txt).lower()
    if any(x in txt for x in ['communic']):
        flag = 1
    else:
        flag = 0
    return flag

In [623]:
def team_player(txt):
    txt = str(txt).lower()
    if any(x in txt for x in ['team']):
        flag = 1
    else:
        flag = 0
    return flag

In [624]:
def leadership_mention(txt):
    txt = str(txt).lower()
    if any(x in txt for x in ['lead']):
        flag = 1
    else:
        flag = 0
    return flag

In [60]:
def containskeywords(txt):
    txt = str(txt).lower()
    flag = 0 
    #if re.search(r"\bsport", txt):
    if any(x in txt for x in ['cashier']):
    #if any(x in txt for x in ['volun']) and any(x in txt for x in ['sport']):
        flag = 1
    else:
        flag = 0
    return flag

In [61]:
data['flag'] = data.resume_text.apply(lambda x:containskeywords(x))
data[data['flag']==1]['resume_text'].sample(n=1).iloc[0]

' amal.allouch.aa@gmail.com contact #: 647-234-9968 Amal Allouch Objective On a mission to obtain a customer service position, where my proven and developing skills in the retail industry allows me to contribute to the goals of your organization while seeking responsibility, trust and the opportunity for growth and career advancement for successful achievements. In addition to that, highly recognized to be efficient, goal oriented and fully dedicated to high levels of customer satisfaction and meeting aggressive company goals. Skills Ability to multitask independently and efficiently in fast paced environment to meet important deadlines and expectations Self-directed, detail oriented, consistent, reliable, and motivated to strive for proficient accuracy and success Approach new tasks with high level of confidence and positive attitude Excellent display of professionalism, and cultural sensitivity Natural ability to communicate effectively to maximize accuracy in client satisfaction Pro

In [62]:
data.columns

Index(['employee_code', 'raw_resume', 'resume_text', 'resume_bline',
       'clean_text', 'hp_class', 'flag'],
      dtype='object')

In [63]:
graph_data = data[['employee_code','flag','hp_class']]

In [64]:
graph_data.groupby('flag').agg({'employee_code':"count",'hp_class':'mean'}).sort_values(by="employee_code",ascending = False)

,employee_code,hp_class
flag,,
0,222,0.261261
1,66,0.181818


### Generating Feature Output

In [628]:
data['no_lang_spoken'] = data.resume_text.apply(lambda x:identify_languages(x)['no_lang'])
data['trilingual_flag'] = data.resume_text.apply(lambda x:identify_languages(x)['trilingual_flag'])
data['goal_record'] = data.resume_text.apply(lambda x: goal_record(x))
data['sales_customer_base_exp'] = data.resume_text.apply(lambda x: sales_customer_base_exp(x))
data['volunteer_exp'] = data.resume_text.apply(lambda x: volunteer_exp(x))
data['problem_solver'] = data.resume_text.apply(lambda x: problem_solver(x))
data['sports_mention'] = data.resume_text.apply(lambda x: sports_mention(x))
data['communication_skills'] = data.resume_text.apply(lambda x: communication_skills(x))
data['team_player'] = data.resume_text.apply(lambda x: team_player(x))
data['leadership_mention'] = data.resume_text.apply(lambda x: leadership_mention(x))

In [629]:
data.columns

Index(['employee_code', 'raw_resume', 'resume_text', 'resume_bline',
       'clean_text', 'hp_class', 'flag', 'no_lang_spoken', 'trilingual_flag',
       'goal_record', 'sales_customer_base_exp', 'volunteer_exp',
       'problem_solver', 'sports_mention', 'communication_skills',
       'team_player', 'leadership_mention'],
      dtype='object')

In [630]:
output_data = data.drop(['raw_resume','hp_class','resume_text','resume_bline','clean_text','flag'],axis =1)

In [632]:
output_data.to_csv('../../../../data/clean_data/features_02_training_df.csv')

In [634]:
output_data.columns

Index(['employee_code', 'no_lang_spoken', 'trilingual_flag', 'goal_record',
       'sales_customer_base_exp', 'volunteer_exp', 'problem_solver',
       'sports_mention', 'communication_skills', 'team_player',
       'leadership_mention'],
      dtype='object')

In [1]:
features_plot = {
    "no_lang_spoken": "Number of Languages Spoken",
    "trilingual_flag": "Trilingual+ Flag",
    "goal_record": "Goal Record History",
    "sales_customer_base_exp": "Customer & Sales Experience",
    "volunteer_exp": "Volunteer Experiece",
    "problem_solver": "Problem Solver" , 
    "sports_mention": "Sports Mention",
    "communication_skills": "Communication Skills Mention",
    "team_player": "Team Player",
    "leadership_mention": "Leadership Mention"
}

In [ ]:
for k in D1.keys():
   print (k, D1[k])

In [593]:
plots = list()
for k in features_plot.keys():
    #print(k)
    #print(features_plot[k])
    
    range_color = ["#17becf","green"]

    col_name = k+":O"
    nice_col_name = features_plot[k]
    
    a = alt.Chart(output_data).mark_bar().encode(
    alt.X(col_name, title=nice_col_name),
    alt.Y('count()', title='count')
        ).properties(title = "Sample Distribution")
    
    b = alt.Chart(output_data).mark_bar().encode(
    alt.X(col_name, title=nice_col_name),
    alt.Y('mean(hp_class):Q', title='High Performer Rate'),
    color = alt.Color('mean(hp_class)',legend=alt.Legend(title = "High Performance Rate"),scale=alt.Scale(domain=(0,0.6), range=range_color) )
        ).properties(title = "High Performance Rate per Value")
    
    c= a|b
    plots.append(c)  

In [602]:
len(plots)

10

In [610]:
plots[0]&plots[1]&plots[2]&plots[3]&plots[4]&plots[5]&plots[6]&plots[7]&plots[8]&plots[9]

alt.VConcatChart(...)